In [245]:
import pandas as pd
import numpy as np
import requests
import os
from dotenv import find_dotenv, load_dotenv
from datetime import datetime
import json
import seaborn as sns
from typing import List, Optional
import quandl
import yfinance as yf
import praw
import plotly.express as px
sns.set_theme(style="darkgrid")

# Load the '.env' file with environment variables
env_loc = find_dotenv('.env')
load_dotenv(env_loc)

True

In [209]:
# Environment variables
# 1. Reddit
reddit_client = os.environ.get('REDDIT_CLIENT')
reddit_secret = os.environ.get('REDDIT_SECRET')
user_agent = 'python- wsb algo for fun by /u/dudik-bender'

# Quandl
quandl_key = os.environ.get('QUANDL_KEY')

In [281]:
## Set up the Reddit API
reddit = praw.Reddit(client_id=reddit_client,
                     client_secret=reddit_secret,
                     user_agent=user_agent)

In [286]:
comments = []
for post in reddit.subreddit('wallstreetbets').top(limit=5):
    comments.append(post.comments)

In [273]:
# Quandl Python Package (https://docs.quandl.com/docs/python-installation)
quandl.ApiConfig.api_key = quandl_key # Set your Quandl API key to make calls (otherwise, limit is 50 per day)

today = datetime.today().strftime('%y-%m-%d') # Reformats today's date into the proper structure for Quandl's API

# Package returns the data in Pandas DataFrame
df = quandl.get('FINRA/FNYX_GME', start_date='2019-12-31', end_date=today)

In [280]:
normalise = True
column = 'TotalVolume'

def normalise_column(dataframe:object, normalise_column: str):
        first_value = dataframe.iloc[0][normalise_column]
        dataframe['Normalised'] = [ (x - first_date_value) / first_date_value for x in df[normalise_column] ]
        return dataframe

if (normalise == True) & (normalise_column is not None):
    df = normalise_column(df, column)

df

,ShortVolume,ShortExemptVolume,TotalVolume,normalised,Normalised
Date,,,,,
2019-12-31,242075.0,100.0,412620.0,0.000000,0.000000
2020-01-02,276386.0,2448.0,527414.0,0.278208,0.278208
2020-01-03,306258.0,2.0,506189.0,0.226768,0.226768
2020-01-06,198834.0,15.0,319375.0,-0.225983,-0.225983
2020-01-07,351972.0,991.0,523321.0,0.268288,0.268288
...,...,...,...,...,...
2021-01-25,6034347.0,711.0,19909430.0,47.251248,47.251248
2021-01-26,7158779.0,699.0,25065384.0,59.746895,59.746895
2021-01-27,3690271.0,6081.0,7140181.0,16.304496,16.304496


In [278]:
gme = yf.Ticker("GME")
hist = gme.history(start='2020-01-01')[['Open','High','Low','Close']]
hist.reset_index()

,Date,Open,High,Low,Close
0,2020-01-02,6.140000,6.470000,6.070000,6.310000
1,2020-01-03,6.210000,6.250000,5.840000,5.880000
2,2020-01-06,5.800000,5.910000,5.600000,5.850000
3,2020-01-07,5.770000,5.830000,5.440000,5.520000
4,2020-01-08,5.490000,5.850000,5.410000,5.720000
...,...,...,...,...,...
267,2021-01-25,96.730003,159.179993,61.130001,76.790001
268,2021-01-26,88.559998,150.000000,80.199997,147.979996
269,2021-01-27,354.829987,380.000000,249.000000,347.510010
270,2021-01-28,265.000000,483.000000,112.250000,193.600006


In [275]:
gme.info#['floatShares']

{'zip': '76051',
 'sector': 'Consumer Cyclical',
 'fullTimeEmployees': 14000,
 'longBusinessSummary': 'GameStop Corp. operates as a multichannel video game, consumer electronics, and collectibles retailer in the United States, Canada, Australia, and Europe. The company sells new and pre-owned video game platforms; accessories, including controllers, gaming headsets, virtual reality products, and memory cards; new and pre-owned video game software; and in-game digital currency, digital downloadable content, and full-game downloads, as well as network points cards, and prepaid digital and prepaid subscription cards. It also sells collectibles comprising licensed merchandise primarily related to the video game, television, and movie industries, as well as pop culture themes. The company operates its stores and e-commerce sites under the GameStop, EB Games, and Micromania brands; and collectibles stores under the Zing Pop Culture and ThinkGeek brand, as well as offers Game Informer, a prin

In [261]:
merged = hist.merge(dfs, on='Date')

In [262]:
merged.reset_index().groupby('Date').sum()
merged['Float'] = gme.info['floatShares']
merged.drop_duplicates()

,Open,High,Low,Close,ShortVolume,ShortExemptVolume,TotalVolume,Float
Date,,,,,,,,
2020-01-02,6.140000,6.470000,6.070000,6.310000,1148513.0,10920.0,1713318.0,46888789
2020-01-03,6.210000,6.250000,5.840000,5.880000,826414.0,12001.0,1338436.0,46888789
2020-01-06,5.800000,5.910000,5.600000,5.850000,697622.0,20584.0,1178801.0,46888789
2020-01-07,5.770000,5.830000,5.440000,5.520000,956826.0,7363.0,2431645.0,46888789
2020-01-08,5.490000,5.850000,5.410000,5.720000,1276856.0,26196.0,2333823.0,46888789
...,...,...,...,...,...,...,...,...
2021-01-25,96.730003,159.179993,61.130001,76.790001,27264073.0,393941.0,72037408.0,46888789
2021-01-26,88.559998,150.000000,80.199997,147.979996,27006738.0,514375.0,81988763.0,46888789
2021-01-27,354.829987,380.000000,249.000000,347.510010,15427666.0,161898.0,28265535.0,46888789


In [290]:
short_interest = merged[['TotalVolume','Float']]
short_interest.columns = ['Short Interest','Current Float']
stock = 'GME'
short_interest_fig = px.line(short_interest, title=f'Short Interest vs. Current Available Shares for ${stock}')
short_interest_fig.show()

In [297]:
hourly = gme.history(start='2021-01-20', interval='2m')[['Open','High','Low','Close']]
fig = px.line(data_frame=hourly[['High','Low']], title=f'Daily Movement for ${stock}')
fig.show()

In [ ]:
## NEXT STEPS
## Find mentions of the stock
## Sentiment analysis
## Scores and trending